In [2]:
import scipy.signal as signal
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [3]:
'''
算术平均滤波法
'''
def ArithmeticAverage(inputs,per):
	if np.shape(inputs)[0] % per != 0:
		lengh = np.shape(inputs)[0] / per
		for x in range(int(np.shape(inputs)[0]),int(lengh + 1)*per):
			inputs = np.append(inputs,inputs[np.shape(inputs)[0]-1])
	inputs = inputs.reshape((-1,per))
	mean = []
	for tmp in inputs:
		mean.append(tmp.mean())
	return mean

In [4]:
'''
递推平均滤波法
'''
def SlidingAverage(inputs,per):
	if np.shape(inputs)[0] % per != 0:
		lengh = np.shape(inputs)[0] / per
		for x in range(int(np.shape(inputs)[0]),int(lengh + 1)*per):
			inputs = np.append(inputs,inputs[np.shape(inputs)[0]-1])
	inputs = inputs.reshape((-1,per))
	tmpmean = inputs[0].mean()
	mean = []
	for tmp in inputs:
		mean.append((tmpmean+tmp.mean())/2)
		tmpmean = tmp.mean()
	return mean

In [5]:
'''
中值平均滤波法
'''
def MedianAverage(inputs,per):
	if np.shape(inputs)[0] % per != 0:
		lengh = np.shape(inputs)[0] / per
		for x in range(int(np.shape(inputs)[0]),int(lengh + 1)*per):
			inputs = np.append(inputs,inputs[np.shape(inputs)[0]-1])
	inputs = inputs.reshape((-1,per))
	mean = []
	for tmp in inputs:
		tmp = np.delete(tmp,np.where(tmp==tmp.max())[0],axis = 0)
		tmp = np.delete(tmp,np.where(tmp==tmp.min())[0],axis = 0)
		mean.append(tmp.mean())
	return mean

In [6]:
'''
限幅平均滤波法
Amplitude:	限制最大振幅
'''
def AmplitudeLimitingAverage(inputs,per,Amplitude):
	if np.shape(inputs)[0] % per != 0:
		lengh = np.shape(inputs)[0] / per
		for x in range(int(np.shape(inputs)[0]),int(lengh + 1)*per):
			inputs = np.append(inputs,inputs[np.shape(inputs)[0]-1])
	inputs = inputs.reshape((-1,per))
	mean = []
	tmpmean = inputs[0].mean()
	tmpnum = inputs[0][0]						#上一次限幅后结果
	for tmp in inputs:
		for index,newtmp in enumerate(tmp):
			if np.abs(tmpnum-newtmp) > Amplitude:
				tmp[index] = tmpnum
			tmpnum = newtmp
		mean.append((tmpmean+tmp.mean())/2)
		tmpmean = tmp.mean()
	return mean

In [7]:
'''
一阶滞后滤波法
a:			滞后程度决定因子，0~1
'''
def FirstOrderLag(inputs,a):
	tmpnum = inputs[0]							#上一次滤波结果
	for index,tmp in enumerate(inputs):
		inputs[index] = (1-a)*tmp + a*tmpnum
		tmpnum = tmp
	return inputs

In [8]:
'''
加权递推平均滤波法
'''
def WeightBackstepAverage(inputs,per):
	weight = np.array(range(1,np.shape(inputs)[0]+1))			#权值列表
	weight = weight/weight.sum()
 
	for index,tmp in enumerate(inputs):
		inputs[index] = inputs[index]*weight[index]
	return inputs

In [9]:
'''
消抖滤波法
N:			消抖上限
'''
def ShakeOff(inputs,N):
	usenum = inputs[0]								#有效值
	i = 0 											#标记计数器
	for index,tmp in enumerate(inputs):
		if tmp != usenum:					
			i = i + 1
			if i >= N:
				i = 0
				inputs[index] = usenum
	return inputs

In [10]:
'''
限幅消抖滤波法
Amplitude:	限制最大振幅
N:			消抖上限
'''
def AmplitudeLimitingShakeOff(inputs,Amplitude,N):
	#print(inputs)
	tmpnum = inputs[0]
	for index,newtmp in enumerate(inputs):
		if np.abs(tmpnum-newtmp) > Amplitude:
			inputs[index] = tmpnum
		tmpnum = newtmp
	#print(inputs)
	usenum = inputs[0]
	i = 0
	for index2,tmp2 in enumerate(inputs):
		if tmp2 != usenum:
			i = i + 1
			if i >= N:
				i = 0
				inputs[index2] = usenum
	#print(inputs)
	return inputs

In [11]:
def gradient(inputs):
    result = []
    prev_val = 0
    for val in inputs:
        result.append(val - prev_val)
        prev_val = val
    return result

In [12]:
def gradient_filter(inputs, min_gradient, max_gradient):
    result = []
    prev_val = 0
    for val in inputs:
        gradient_val = val - prev_val
        if gradient_val > min_gradient and gradient_val < max_gradient:
            result.append(val)
            prev_val = val
        else:
            result.append(prev_val)
    return result

In [13]:
all_data = pd.read_csv('RAIL-speed.csv')
all_data

FileNotFoundError: [Errno 2] No such file or directory: 'RAIL-speed.csv'

In [14]:
series = all_data['R0']

%matplotlib
plt.figure(figsize=(20, 10))
plt.subplot(3,1,1)
plt.plot(series)
plt.show()

gradient_result = gradient(series)
plt.subplot(3,1,2)
plt.plot(gradient_result)

result = gradient_filter(series, -0.5, 7)
plt.subplot(3,1,3)
plt.plot(result)
plt.show()

NameError: name 'all_data' is not defined